# Forest Loss Prediction Modeling

This notebook implements three regression models to predict tree cover loss:
1. **Linear Regression** - Baseline model for comparison
2. **Random Forest Regression** - Handles nonlinear effects & interactions
3. **XGBoost** - High accuracy for structured environmental data

**Evaluation Metrics**: R² (Coefficient of Determination) and RMSE (Root Mean Squared Error)

In [21]:
## 1️⃣ Import Libraries and Load Data

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv("../data/processed/merged_clean_data.csv")
print(f"✅ Loaded dataset: {df.shape}")
print(f"   Columns: {len(df.columns)}")
df.head()


✅ Loaded dataset: (31873, 24)
   Columns: 24


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,settlements_infrastructure,shifting_cultivation,wildfire,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,,,...,,,,,,...,,,,NaN
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,,,...,,,,,,...,,,,NaN
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,,,...,,,,,,...,,,,NaN
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,,,...,,,,,,...,,,,NaN
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,,,...,,,,,,...,,,,NaN


## 2️⃣ Data Preprocessing and Feature Engineering


In [22]:
# Clean column names
def clean_columns_names(df):
    df.columns = df.columns.str.replace(" ", "")
    df = df.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
    return df


# Handle missing data intelligently
def handle_missing_data(df):
    """Handle missing data instead of dropping rows."""
    df = df.copy()
    
    # Check required columns exist
    required_cols = ['country', 'threshold', 'year', 'tree_cover_loss_ha']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Driver data: fill with 0 (no driver data = no loss attributed to drivers)
    driver_cols = ['hard_commodities', 'logging', 'wildfire', 'permanent_agriculture', 
                   'shifting_cultivation', 'settlements_infrastructure', 'other_natural_disturbances']
    driver_cols = [c for c in driver_cols if c in df.columns]
    if driver_cols:
        # Convert to numeric, replacing non-numeric values with NaN, then fill with 0
        for col in driver_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df[driver_cols] = df[driver_cols].fillna(0)
    
    # Carbon data: impute with median
    carbon_cols = ['carbon_gross_emissions_MgCO2e', 'gfw_forest_carbon_net_flux__mg_co2e_yr-1']
    carbon_cols = [c for c in carbon_cols if c in df.columns]
    if carbon_cols:
        # Convert to numeric first (handles empty strings, etc.)
        for col in carbon_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Only impute if there are numeric values
        numeric_carbon_cols = [c for c in carbon_cols if df[c].dtype in ['float64', 'int64']]
        if numeric_carbon_cols:
            imputer = SimpleImputer(strategy='median')
            df[numeric_carbon_cols] = imputer.fit_transform(df[numeric_carbon_cols])
    
    # Primary forest: forward fill within country-threshold groups
    if 'primary_forest_loss_ha' in df.columns:
        # Convert to numeric first
        df['primary_forest_loss_ha'] = pd.to_numeric(df['primary_forest_loss_ha'], errors='coerce')
        
        sort_cols = [c for c in ['country', 'threshold', 'year'] if c in df.columns]
        if len(sort_cols) > 0:
            df = df.sort_values(sort_cols)
            group_cols = [c for c in ['country', 'threshold'] if c in df.columns]
            if len(group_cols) > 0:
                df['primary_forest_loss_ha'] = df.groupby(group_cols)['primary_forest_loss_ha'].fillna(method='ffill').fillna(0)
    
    return df

# Create temporal features
def create_temporal_features(df):
    """Create time-based and lag features."""
    df = df.copy()
    
    # Check required columns
    if 'tree_cover_loss_ha' not in df.columns:
        print("⚠️ Warning: 'tree_cover_loss_ha' not found, skipping temporal features")
        return df
    
    # Ensure target column is numeric
    df['tree_cover_loss_ha'] = pd.to_numeric(df['tree_cover_loss_ha'], errors='coerce')
    
    # Determine grouping columns
    group_cols = []
    for col in ['country', 'threshold']:
        if col in df.columns:
            group_cols.append(col)
    
    if len(group_cols) == 0:
        print("⚠️ Warning: No grouping columns found, skipping temporal features")
        return df
    
    # Sort by available columns
    sort_cols = group_cols + ['year'] if 'year' in df.columns else group_cols
    if len(sort_cols) > 0:
        df = df.sort_values(sort_cols)
    
    # Lag features (previous year's loss)
    try:
        df['loss_lag1'] = df.groupby(group_cols)['tree_cover_loss_ha'].shift(1)
    except Exception as e:
        print(f"⚠️ Warning: Could not create loss_lag1: {e}")
        df['loss_lag1'] = np.nan
    
    # Rolling averages
    try:
        df['loss_rolling_mean_3y'] = df.groupby(group_cols)['tree_cover_loss_ha'].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean()
        )
    except Exception as e:
        print(f"⚠️ Warning: Could not create loss_rolling_mean_3y: {e}")
        df['loss_rolling_mean_3y'] = np.nan
    
    # Years since 2000
    if 'year' in df.columns:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df['years_since_2000'] = df['year'] - 2000
    
    # Remaining forest
    try:
        df['cumulative_loss'] = df.groupby(group_cols)['tree_cover_loss_ha'].cumsum()
        if 'extent_2000_ha' in df.columns:
            df['extent_2000_ha'] = pd.to_numeric(df['extent_2000_ha'], errors='coerce')
            df['remaining_forest'] = (df['extent_2000_ha'] - df['cumulative_loss']).clip(lower=0)
            df['remaining_forest_pct'] = (df['remaining_forest'] / (df['extent_2000_ha'] + 1) * 100).clip(0, 100)
    except Exception as e:
        print(f"⚠️ Warning: Could not create remaining forest features: {e}")
        if 'cumulative_loss' not in df.columns:
            df['cumulative_loss'] = np.nan
        if 'remaining_forest' not in df.columns:
            df['remaining_forest'] = np.nan
        if 'remaining_forest_pct' not in df.columns:
            df['remaining_forest_pct'] = np.nan
    
    return df

# Apply preprocessing
print("Processing data...")
print(f"Initial columns: {list(df.columns)[:10]}...")  # Show first 10 columns
df = clean_columns_names(df)
df = handle_missing_data(df)
df = create_temporal_features(df)
print(f"✅ Processed dataset: {df.shape}")
df.head()

Processing data...
Initial columns: ['country                          ', 'threshold ', 'area_ha_x    ', 'extent_2000_ha ', 'extent_2010_ha ', 'gain_2000-2012_ha ', 'tree_cover_loss_ha ', 'year ', 'area_ha_y   ', 'primary_forest_loss_ha ']...
✅ Processed dataset: (31873, 30)


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e,loss_lag1,loss_rolling_mean_3y,years_since_2000,cumulative_loss,remaining_forest,remaining_forest_pct
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,,0.0,...,,,-3203279.5,3004087.5,NaN,103.000000,1,103.0,64383552.0,99.999838
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,,0.0,...,,,-3203279.5,3004087.5,103.0,158.500000,2,317.0,64383338.0,99.999506
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,,0.0,...,,,-3203279.5,3004087.5,214.0,194.666667,3,584.0,64383071.0,99.999091
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,,0.0,...,,,-3203279.5,3004087.5,267.0,235.666667,4,810.0,64382845.0,99.998740
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,,0.0,...,,,-3203279.5,3004087.5,226.0,253.666667,5,1078.0,64382577.0,99.998324


## 3️⃣ Feature Selection and Data Splitting


In [23]:
# Define target and features
target = "tree_cover_loss_ha"
features = [
    "year", "years_since_2000",
    "threshold",
    "loss_lag1", "loss_rolling_mean_3y",
    "hard_commodities", "logging", "wildfire",
    "permanent_agriculture", "shifting_cultivation",
    "gfw_forest_carbon_net_flux__mg_co2e_yr-1",
    "carbon_gross_emissions_MgCO2e",
    "primary_forest_loss_ha",
    "umd_tree_cover_extent_2000__ha",
    "gfw_aboveground_carbon_stocks_2000__mg_c"
]

# Filter to available features
available_features = [f for f in features if f in df.columns]
print(f"Using {len(available_features)} features: {available_features}")

# Remove rows with missing target
df = df.dropna(subset=[target, "country"])

# Ensure target is numeric
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna(subset=[target])  # Remove rows where target couldn't be converted

# Encode country
df_enc = pd.get_dummies(df, columns=["country"], drop_first=True)
X = df_enc[available_features + [c for c in df_enc.columns if c.startswith("country_")]]
y = df_enc[target]

# Convert all feature columns to numeric (handles empty strings, etc.)
print("\nConverting all features to numeric...")
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Fill any remaining NaN values in features with 0 (for missing temporal features)
X = X.fillna(0)

# Ensure y is numeric
y = pd.to_numeric(y, errors='coerce')
y = y.fillna(0)

print(f"\n✅ Final dataset shape: {X.shape}")
print(f"   Target variable: {target}")
print(f"   Feature dtypes: {X.dtypes.value_counts().to_dict()}")

# Time-based split (important for time series data)
# Train: 2001-2019, Test: 2020-2024
train_mask = df_enc['year'] <= 2019
test_mask = df_enc['year'] >= 2020

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]

# Final check: ensure all are numeric
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)

print(f"\n📊 Data Split:")
print(f"   Train: {len(X_train):,} samples (2001-2019)")
print(f"   Test:  {len(X_test):,} samples (2020-2024)")
print(f"   X_train dtype: {X_train.dtypes.value_counts().to_dict()}")
print(f"   y_train dtype: {y_train.dtype}")


Using 15 features: ['year', 'years_since_2000', 'threshold', 'loss_lag1', 'loss_rolling_mean_3y', 'hard_commodities', 'logging', 'wildfire', 'permanent_agriculture', 'shifting_cultivation', 'gfw_forest_carbon_net_flux__mg_co2e_yr-1', 'carbon_gross_emissions_MgCO2e', 'primary_forest_loss_ha', 'umd_tree_cover_extent_2000__ha', 'gfw_aboveground_carbon_stocks_2000__mg_c']

Converting all features to numeric...

✅ Final dataset shape: (31872, 180)
   Target variable: tree_cover_loss_ha
   Feature dtypes: {dtype('bool'): 165, dtype('float64'): 12, dtype('int64'): 3}

📊 Data Split:
   Train: 25,232 samples (2001-2019)
   Test:  6,640 samples (2020-2024)
   X_train dtype: {dtype('float64'): 180}
   y_train dtype: float64


## 4️⃣ Model Training and Evaluation

We train three models and compare their performance using R² and RMSE.


In [24]:
# Evaluation function
def evaluate_model(y_true, y_pred, model_name):
    """Evaluate model using R² and RMSE."""
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    print(f"\n{model_name} Performance:")
    print(f"  R² Score:  {r2:.4f}")
    print(f"  RMSE:      {rmse:,.2f} ha")
    
    return {'R²': r2, 'RMSE': rmse}

# Store results
results = {}


### 4.1 Linear Regression (Baseline Model)


In [25]:
# Train Linear Regression baseline
print("=" * 60)
print("Training Linear Regression (Baseline)...")
print("=" * 60)

lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate
y_pred_lr = lr.predict(X_test)
results['Linear Regression'] = evaluate_model(y_test, y_pred_lr, "Linear Regression")


Training Linear Regression (Baseline)...

Linear Regression Performance:
  R² Score:  0.9358
  RMSE:      160,102.05 ha


### 4.2 Random Forest Regression (Nonlinear Effects & Interactions)


In [26]:
# Train Random Forest Regression
print("\n" + "=" * 60)
print("Training Random Forest Regression...")
print("=" * 60)

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    random_state=42,
    n_jobs=-1,
    verbose=0
)
rf.fit(X_train, y_train)

# Evaluate
y_pred_rf = rf.predict(X_test)
results['Random Forest'] = evaluate_model(y_test, y_pred_rf, "Random Forest")



Training Random Forest Regression...

Random Forest Performance:
  R² Score:  0.8834
  RMSE:      215,752.79 ha


### 4.3 XGBoost (High Accuracy for Structured Environmental Data)


In [27]:
# Train XGBoost
print("\n" + "=" * 60)
print("Training XGBoost...")
print("=" * 60)

xgb = XGBRegressor(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)
xgb.fit(X_train, y_train)

# Evaluate
y_pred_xgb = xgb.predict(X_test)
results['XGBoost'] = evaluate_model(y_test, y_pred_xgb, "XGBoost")



Training XGBoost...

XGBoost Performance:
  R² Score:  0.8494
  RMSE:      245,189.40 ha


## 5️⃣ Model Comparison


## 5️⃣ Model Performance Analysis

Let's analyze why Linear Regression might be performing better than tree-based models.


In [28]:
# Diagnostic Analysis: Why Linear Regression Might Perform Better

print("=" * 60)
print("DIAGNOSTIC ANALYSIS")
print("=" * 60)

# 1. Check for potential data leakage
print("\n1. Data Leakage Check:")
leakage_features = ['loss_lag1', 'loss_rolling_mean_3y', 'cumulative_loss']
for feat in leakage_features:
    if feat in X.columns:
        corr = X[feat].corr(y)
        print(f"   {feat}: correlation with target = {corr:.4f}")

# 2. Check feature importance (if available)
print("\n2. Feature Importance (Random Forest):")
if 'rf' in locals():
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    print(f"   Top 10 most important features:")
    for idx, row in feature_importance.head(10).iterrows():
        print(f"      {row['feature']}: {row['importance']:.4f}")

# 3. Check for overfitting
print("\n3. Overfitting Check:")
if 'rf' in locals() and 'xgb' in locals():
    train_r2_rf = r2_score(y_train, rf.predict(X_train))
    test_r2_rf = r2_score(y_test, rf.predict(X_test))
    train_r2_xgb = r2_score(y_train, xgb.predict(X_train))
    test_r2_xgb = r2_score(y_test, xgb.predict(X_test))
    train_r2_lr = r2_score(y_train, lr.predict(X_train))
    test_r2_lr = r2_score(y_test, lr.predict(X_test))
    
    print(f"   Linear Regression:")
    print(f"      Train R²: {train_r2_lr:.4f}, Test R²: {test_r2_lr:.4f}, Gap: {train_r2_lr - test_r2_lr:.4f}")
    print(f"   Random Forest:")
    print(f"      Train R²: {train_r2_rf:.4f}, Test R²: {test_r2_rf:.4f}, Gap: {train_r2_rf - test_r2_rf:.4f}")
    print(f"   XGBoost:")
    print(f"      Train R²: {train_r2_xgb:.4f}, Test R²: {test_r2_xgb:.4f}, Gap: {train_r2_xgb - test_r2_xgb:.4f}")

# 4. Check data characteristics
print("\n4. Data Characteristics:")
print(f"   Number of features: {X.shape[1]}")
print(f"   Number of samples (train): {X_train.shape[0]}")
print(f"   Feature-to-sample ratio: {X_train.shape[0] / X_train.shape[1]:.2f}")
print(f"   Target variable range: [{y.min():,.0f}, {y.max():,.0f}]")
print(f"   Target variable mean: {y.mean():,.0f}")
print(f"   Target variable std: {y.std():,.0f}")

# 5. Check for linear relationships
print("\n5. Linearity Check (Top Correlations):")
correlations = X.corrwith(y).abs().sort_values(ascending=False)
print(f"   Top 10 feature correlations with target:")
for feat, corr in correlations.head(10).items():
    print(f"      {feat}: {corr:.4f}")


DIAGNOSTIC ANALYSIS

1. Data Leakage Check:
   loss_lag1: correlation with target = 0.9139
   loss_rolling_mean_3y: correlation with target = 0.9710

2. Feature Importance (Random Forest):
   Top 10 most important features:
      loss_rolling_mean_3y: 0.9619
      loss_lag1: 0.0167
      year: 0.0060
      years_since_2000: 0.0059
      threshold: 0.0019
      country_Russia: 0.0018
      country_Brazil: 0.0014
      carbon_gross_emissions_MgCO2e: 0.0014
      gfw_forest_carbon_net_flux__mg_co2e_yr-1: 0.0005
      umd_tree_cover_extent_2000__ha: 0.0004

3. Overfitting Check:
   Linear Regression:
      Train R²: 0.9651, Test R²: 0.9358, Gap: 0.0293
   Random Forest:
      Train R²: 0.9978, Test R²: 0.8834, Gap: 0.1144
   XGBoost:
      Train R²: 0.9998, Test R²: 0.8494, Gap: 0.1504

4. Data Characteristics:
   Number of features: 180
   Number of samples (train): 25232
   Feature-to-sample ratio: 140.18
   Target variable range: [0, 10,176,019]
   Target variable mean: 127,563
   Targe

### Possible Reasons Why Linear Regression Performs Better:

1. **Data Leakage from Lag Features**: `loss_lag1` and `loss_rolling_mean_3y` are highly correlated with the target, creating near-perfect predictions
2. **High Dimensionality**: 180 features (165 country dummies) with relatively few samples - linear models can handle this better
3. **Linear Relationships**: The data may have strong linear relationships that tree models can't exploit as efficiently
4. **Overfitting in Tree Models**: Random Forest/XGBoost might be overfitting to training data
5. **Missing Data Handling**: Filling with 0 might affect tree-based models differently than linear models
6. **Feature Scale**: Tree models might struggle with the wide range of target values

**Recommendations:**
- Remove or reduce lag features to prevent data leakage
- Try regularization (Ridge/Lasso) for linear regression
- Tune hyperparameters for tree-based models
- Use feature selection to reduce dimensionality


In [29]:
# Compare all models
print("\n" + "=" * 60)
print("MODEL COMPARISON SUMMARY")
print("=" * 60)

comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.sort_values('R²', ascending=False)

print("\n" + comparison_df.to_string())
print("\n" + "=" * 60)

# Identify best model
best_model_name = comparison_df.index[0]
best_r2 = comparison_df.loc[best_model_name, 'R²']
best_rmse = comparison_df.loc[best_model_name, 'RMSE']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score:  {best_r2:.4f}")
print(f"   RMSE:      {best_rmse:,.2f} ha")



MODEL COMPARISON SUMMARY

                         R²           RMSE
Linear Regression  0.935797  160102.054381
Random Forest      0.883407  215752.787387
XGBoost            0.849421  245189.400146


🏆 Best Model: Linear Regression
   R² Score:  0.9358
   RMSE:      160,102.05 ha


## 6️⃣ Save Best Model


In [30]:
# Select best model
if best_model_name == 'Linear Regression':
    best_model = lr
elif best_model_name == 'Random Forest':
    best_model = rf
else:
    best_model = xgb

# Save best model
model_bundle = {
    "model": best_model,
    "model_name": best_model_name,
    "features": X.columns.tolist(),
    "metrics": {
        "R²": float(best_r2),
        "RMSE": float(best_rmse)
    },
    "comparison": comparison_df.to_dict('index')
}

joblib.dump(model_bundle, "../models/tree_cover_loss_model.pkl")
print(f"\n✅ Saved best model ({best_model_name}) to ../models/tree_cover_loss_model.pkl")

# Also save all models for comparison
all_models = {
    "linear_regression": lr,
    "random_forest": rf,
    "xgboost": xgb,
    "best_model_name": best_model_name,
    "features": X.columns.tolist(),
    "results": results
}
joblib.dump(all_models, "../models/all_models.pkl")
print("✅ Saved all models to ../models/all_models.pkl")



✅ Saved best model (Linear Regression) to ../models/tree_cover_loss_model.pkl
✅ Saved all models to ../models/all_models.pkl


## 7️⃣ Prediction Helper Functions


In [31]:
# Linear extrapolation helper for future predictions
def linear_extrapolate(years, values, target_year):
    """Return linear extrapolation of values to target_year."""
    # Convert to DataFrame
    s = pd.DataFrame({"year": years, "val": values})
    
    # Convert both columns to numeric, coercing errors to NaN
    s["year"] = pd.to_numeric(s["year"], errors='coerce')
    s["val"] = pd.to_numeric(s["val"], errors='coerce')
    
    # Drop NaN values and duplicates
    s = s.dropna().drop_duplicates().sort_values("year")
    
    if len(s) >= 3:
        # Ensure values are float64
        x = s["year"].values.astype(float)
        y = s["val"].values.astype(float)
        
        # Perform linear regression
        A = np.vstack([x, np.ones_like(x)]).T
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a * target_year + b)
    
    # If not enough data, return last value or NaN
    if len(s) > 0:
        return float(s["val"].iloc[-1])
    return np.nan

print("✅ Helper functions defined")


✅ Helper functions defined


## 8️⃣ Predict for Future Years


In [32]:
# Load saved model
bundle = joblib.load("../models/tree_cover_loss_model.pkl")
model = bundle["model"]
feature_cols = bundle["features"]

def predict_tree_loss_future(country, year):
    """Predict tree cover loss for a country and future year."""
    dff = df[df["country"] == country].copy()
    
    # Filter to threshold 30 if available
    if "threshold" in dff.columns:
        filt = dff[dff["threshold"] == 30]
        if not filt.empty:
            dff = filt
    
    if dff.empty:
        print(f"⚠️ No data for {country}")
        return None
    
    # Get latest data point
    latest = dff.sort_values("year").tail(1).copy()
    
    # Extrapolate features to target year
    base_features = available_features.copy()
    for col in base_features:
        if col != "year" and col in dff.columns:
            latest[col] = linear_extrapolate(dff["year"], dff[col], year)
    
    latest["year"] = year
    
    # Encode country and align features
    enc = pd.get_dummies(latest, columns=["country"], drop_first=True)
    for c in feature_cols:
        if c not in enc.columns:
            enc[c] = 0
    
    X_future = enc[feature_cols].fillna(0)
    
    # Predict
    pred = model.predict(X_future)[0]
    print(f"🌲 Predicted Tree-Cover-Loss in {country} ({year}): {pred:,.0f} ha")
    return pred

# Example predictions
print("Example predictions:")
predict_tree_loss_future("Brazil", 2035)
predict_tree_loss_future("Indonesia", 2030)


Example predictions:
🌲 Predicted Tree-Cover-Loss in Brazil (2035): 4,065,720 ha
🌲 Predicted Tree-Cover-Loss in Indonesia (2030): 1,360,369 ha


np.float64(1360368.6072850677)

## 9️⃣ Generate Forecasts for All Countries (2025-2035)


In [33]:
# Generate forecasts for all countries
print("Generating forecasts for 2025-2035...")
future_years = range(df["year"].max() + 1, df["year"].max() + 11)
rows = []

for country in sorted(df["country"].unique()):
    for year in future_years:
        val = predict_tree_loss_future(country, year)
        rows.append({"country": country, "year": year, "pred_tree_loss_ha": val})

pred_df = pd.DataFrame(rows)
pred_df.to_csv("../data/predicted_tree_cover_loss_2025_2035.csv", index=False)
print(f"\n✅ Saved forecast table with {len(pred_df):,} predictions")
print(f"   Countries: {pred_df['country'].nunique()}")
print(f"   Years: {pred_df['year'].min()}-{pred_df['year'].max()}")


Generating forecasts for 2025-2035...
🌲 Predicted Tree-Cover-Loss in Afghanistan (2025): 5,104 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2026): 5,155 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2027): 5,205 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2028): 5,255 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2029): 5,306 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2030): 5,356 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2031): 5,406 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2032): 5,457 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2033): 5,507 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2034): 5,557 ha
🌲 Predicted Tree-Cover-Loss in Albania (2025): 7,524 ha
🌲 Predicted Tree-Cover-Loss in Albania (2026): 7,519 ha
🌲 Predicted Tree-Cover-Loss in Albania (2027): 7,514 ha
🌲 Predicted Tree-Cover-Loss in Albania (2028): 7,509 ha
🌲 Predicted Tree-Cover-Loss in Albania (2029): 7,504 ha
🌲 Predicted Tree-Cover-Loss in Albania (2030): 7,499 ha
🌲 Predicted Tree-Cover-Los

2️⃣ Linear extrapolation helper

In [34]:
## 2️⃣ Helper for future extrapolation
import numpy as np


def linear_extrapolate(years, values, target_year):
    """Return linear extrapolation of `values` to `target_year`."""
    # Convert to DataFrame
    s = pd.DataFrame({"year": years, "val": values})
    
    # Convert both columns to numeric, coercing errors to NaN
    s["year"] = pd.to_numeric(s["year"], errors='coerce')
    s["val"] = pd.to_numeric(s["val"], errors='coerce')
    
    # Drop NaN values and duplicates
    s = s.dropna().drop_duplicates().sort_values("year")
    
    if len(s) >= 3:
        # Ensure values are float64
        x = s["year"].values.astype(float)
        y = s["val"].values.astype(float)
        
        # Perform linear regression
        A = np.vstack([x, np.ones_like(x)]).T
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a * target_year + b)
    
    # If not enough data, return last value or NaN
    if len(s) > 0:
        return float(s["val"].iloc[-1])
    return np.nan


3️⃣ Predict for any (country, year)

In [35]:
## 3️⃣ Predict Tree-Cover-Loss for any country and future year
bundle = joblib.load("../models/tree_cover_loss_model.pkl")
model, feature_cols = bundle["model"], bundle["features"]


def predict_tree_loss_future(country, year):
    dff = df[df["country"] == country].copy()
    if "threshold" in dff.columns:
        filt = dff[dff["threshold"] == 30]
        if not filt.empty:
            dff = filt

    if dff.empty:
        print(f"⚠️ No data for {country}")
        return None

    latest = dff.sort_values("year").tail(1).copy()

    # --- extrapolate all numeric predictors to target year ---
    for col in features:
        if col != "year":
            latest[col] = linear_extrapolate(dff["year"], dff[col], year)

    latest["year"] = year

    # encode country and align features
    enc = pd.get_dummies(latest, columns=["country"], drop_first=True)
    for c in feature_cols:
        if c not in enc.columns:
            enc[c] = 0
    X_future = enc[feature_cols].fillna(0)

    pred = model.predict(X_future)[0]
    print(f"🌲 Predicted Tree-Cover-Loss in {country} ({year}): {pred:,.0f} ha")
    return pred


# Example
predict_tree_loss_future("Brazil", 2035)
predict_tree_loss_future("Indonesia", 2030)


🌲 Predicted Tree-Cover-Loss in Brazil (2035): 4,065,720 ha
🌲 Predicted Tree-Cover-Loss in Indonesia (2030): 1,360,369 ha


np.float64(1360368.6072850677)

In [36]:
## 4️⃣ Forecast 2025–2035 for all countries
future_years = range(df["year"].max() + 1, df["year"].max() + 11)
rows = []

for country in sorted(df["country"].unique()):
    for year in future_years:
        val = predict_tree_loss_future(country, year)
        rows.append({"country": country, "year": year, "pred_tree_loss_ha": val})

pred_df = pd.DataFrame(rows)
pred_df.to_csv("../data/predicted_tree_cover_loss_2025_2035.csv", index=False)
print("✅ Saved forecast table.")

🌲 Predicted Tree-Cover-Loss in Afghanistan (2025): 5,104 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2026): 5,155 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2027): 5,205 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2028): 5,255 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2029): 5,306 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2030): 5,356 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2031): 5,406 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2032): 5,457 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2033): 5,507 ha
🌲 Predicted Tree-Cover-Loss in Afghanistan (2034): 5,557 ha
🌲 Predicted Tree-Cover-Loss in Albania (2025): 7,524 ha
🌲 Predicted Tree-Cover-Loss in Albania (2026): 7,519 ha
🌲 Predicted Tree-Cover-Loss in Albania (2027): 7,514 ha
🌲 Predicted Tree-Cover-Loss in Albania (2028): 7,509 ha
🌲 Predicted Tree-Cover-Loss in Albania (2029): 7,504 ha
🌲 Predicted Tree-Cover-Loss in Albania (2030): 7,499 ha
🌲 Predicted Tree-Cover-Loss in Albania (2031): 7,494 ha
🌲 Predic